In [1]:
import requests
import pandas as pd
import time

In [2]:
# Set max columns and rows displayed
pd.set_option('display.max_columns', 10000)
pd.set_option('display.max_rows', 10000)

In [3]:
# initial url for reddit API submissions endpoint
url = 'https://api.pushshift.io/reddit/search/submission'

In [4]:
def pull_data(subreddit, loops):
    params = {
        'subreddit' : subreddit,
        'size' : 500
    }

    # Initial API call
    res = requests.get(url, params)
    data = res.json()
    posts = data['data']

    # Create dataframe
    df = pd.DataFrame(posts)

    # adapted from Jake Ellena - establish earliest post in data and loop, using min_date to replace "before"
    min_date = df['created_utc'].min()

    for i in range(loops - 1):
        params = {
        'subreddit' : subreddit,
        'size' : 500,
        'before' : min_date
    }
        res = requests.get(url, params)
        data = res.json()
        posts = data['data']
        df2 = pd.DataFrame(posts)
        df = pd.concat([df, df2], sort=False)
        min_date = df['created_utc'].min()

    # trim down to just the rows I want
    df = df[['subreddit', 'selftext', 'title', 'created_utc', 'score']]
    
    return df

In [5]:
# call function on exmormon
exmormon = pull_data('exmormon', 20)

In [6]:
# call function on latterdaysaints
latterdaysaints = pull_data('latterdaysaints', 20)

In [7]:
# stick the results together
posts = pd.concat([latterdaysaints, exmormon])

In [8]:
# write results to csv
posts.to_csv('../data/posts.csv', index=False)